In [1]:
from koebe.geometries.euclidean2 import *
from random import random

def areaOfTriangle(A:PointE2, B:PointE2, C:PointE2):
    """Signed area of a triangle"""
    return 0.5 * ((B.x - A.x) * (C.y - A.y) - (C.x - A.x) * (B.y - A.y))

def leftHandTurn(A:PointE2, B:PointE2, C:PointE2):
    """Returns true iff. the points A, B, C constitute a left hand turn."""
    return areaOfParallelogram(A, B, C) > 0

def perturb(pointSet, scale=1e-8):
    """Randomly perturbs a list of points by a small amount."""
    eps = [((random()-0.5)*scale, (random()-0.5)*scale) for _ in range(len(pointSet))]
    return [PointE2(pointSet[i].x + eps[i][0], pointSet[i].y + eps[i][1]) for i in range(len(pointSet))]

class Vertex:
    """
    A class for a Vertex in a Polygon. Each vertex stores a PointE2 .pos for its
    coordinates, as well as .next and .prev attributes for the next and previous 
    vertices along the polygon.
    """
    def __init__(self, pos: PointE2):
        """
        Initializes the vertex at the given position.
        
        Attributes:
            pos: PointE2 - The position to initialize this vertex to.
        """
        self.pos = pos
        self.prev = None
        self.next = None
    
    def setNextVertex(self, nextVertex: "Vertex"):
        """
        Makes nextVertex the next vertex from this one and makes this vertex
        the previous vertex to nextVertex. 
        """
        self.next = nextVertex
        nextVertex.prev = self

class Polygon:
    """
    Simple container class for a Polygon. Should store a firstVertex. 
    As a convenience a list of PointE2 objects may be passed in to the 
    vertexPositions parameter, and the constructor will initialize a 
    doubly-connected linked list of Vertex objects with the PointE2s as the
    .pos attributes. 
    """
    def __init__(self, vertexPositions = []):
        if len(vertexPositions) > 0:
            vertices = [Vertex(p) for p in vertexPositions]
            for i in range(len(vertices)):
                vertices[i - 1].setNextVertex(vertices[i])
            self.firstVertex = vertices[0]
        else:
            self.firstVertex = None
    
    def __iter__(self):
        """
        An iterator for the polygon that allows you to loop
        over a polygon's vertices using: 
        
        for vertex in polygon:
            ... do something with the vertex object ...
        """
        current = self.firstVertex
        while True:
            yield current
            current = current.next
            if current is self.firstVertex:
                break
    
    def vertexPositions(self):
        """
        Returns the positions of all ther vertices in order.
        """
        return [v.pos for v in self] # Uses the __iter__ to build a list
    
    def copy(self):
        """
        Returns a copy of this polygon. This performs a shallow copy
        since PointE2 objects are immutable. 
        """
        return Polygon(self.vertexPositions())

def findLowerTangent(poly, p):
    for q in poly:
        if (leftHandTurn(q.prev.pos, q.pos, p) 
            and leftHandTurn(q.next.pos, q.pos, p)):
            return q
    return None

def findUpperTangent(poly, p):
    for q in poly:
        if (leftHandTurn(p, q.pos, q.next.pos) 
            and leftHandTurn(p, q.pos, q.prev.pos)):
            return q
    return None

def orient(tri):
    if leftHandTurn(*tri):
        return tri
    else:
        return list(reversed(tri))

def incrConv(P):
    """
    Computes the convex hull of the point set P using the incremental algorithm. 
    
    Assumes P has at least three points in it. 
    """
    thePoints = list(P)
    thePoints.sort(key = lambda p : (p.x, p.y))
    poly = Polygon(orient(thePoints[0:3]))
    for p in thePoints[3:]:
        lowerTangent = findLowerTangent(poly, p)
        upperTangent = findUpperTangent(poly, p)
        if lowerTangent == None or upperTangent == None:
            return poly
        pVert = Vertex(p)
        lowerTangent.setNextVertex(pVert)
        pVert.setNextVertex(upperTangent)
        poly.firstVertex = pVert
    return poly

def incrConvAnim(P):
    """
    Computes the convex hull of the point set P using the incremental algorithm. 
    
    Assumes P has at least three points in it. 
    """
    thePoints = list(P)
    thePoints.sort(key = lambda p : (p.x, p.y))
    polys = [Polygon(orient(thePoints[0:3]))]
    chains = [[]]
    for p in thePoints[3:]:
        poly = polys[-1].copy()
        polys.append(poly)
        lowerTangent = findLowerTangent(poly, p)
        upperTangent = findUpperTangent(poly, p)
        
        chain = [lowerTangent]
        while chain[-1] != upperTangent:
            chain.append(chain[-1].next)
        chains.append([v.pos for v in chain])
        
        if lowerTangent == None or upperTangent == None:
            return poly
        pVert = Vertex(p)
        lowerTangent.setNextVertex(pVert)
        pVert.setNextVertex(upperTangent)
        poly.firstVertex = pVert
    return polys, chains

In [22]:
from koebe.geometries.euclidean2 import PointE2

from ipycanvas import Canvas, hold_canvas

def PointEditorCanvas(size, pts, draw_func = None):
    
    points = list(pts)
    canvas = Canvas(size=size)
    selectedPointIdx = -1
    
    def _canvas_draw():
        nonlocal canvas, points, selectedPointIdx, draw_func
        
        with hold_canvas(canvas):
            canvas.clear()

            if draw_func != None:
                draw_func(canvas, points)

            canvas.fill_style = 'blue'
            canvas.fill_rects([round(p.x) - 4 for p in points], 
                              [round(p.y) - 4 for p in points], 
                              8)
            if selectedPointIdx != -1:
                canvas.fill_style = 'red'
                canvas.fill_rect(round(points[selectedPointIdx].x) - 4, 
                                 round(points[selectedPointIdx].y) - 4, 
                                 8)
            
    def handle_mouse_down(x, y):
        nonlocal selectedPointIdx
        # See if any point is close to x, y
        cursorPoint = PointE2(x, y)
        sqDists = [p.distSqTo(cursorPoint) for p in points]
        minIdx = sqDists.index(min(sqDists))
        if sqDists[minIdx] < 24:
            selectedPointIdx = minIdx
        _canvas_draw()

    def handle_mouse_up(x, y):
        nonlocal selectedPointIdx
        selectedPointIdx = -1 # No point is selected anymore.
        _canvas_draw()

    def handle_mouse_move(x, y):
        nonlocal selectedPointIdx
        if selectedPointIdx >= 0:
            points[selectedPointIdx] = PointE2(x, y)
        _canvas_draw()
    
    canvas.on_mouse_down(handle_mouse_down)
    canvas.on_mouse_up(handle_mouse_up)
    canvas.on_mouse_move(handle_mouse_move)
    
    _canvas_draw()
    
    return canvas

def draw(canvas, points):
    
    convHull = incrConv(perturb(points))

    canvas.begin_path()
    canvas.move_to(round(convHull.firstVertex.pos.x), round(convHull.firstVertex.pos.y))
    for p in convHull.vertexPositions():
        canvas.line_to(round(p.x), round(p.y))
    canvas.close_path()
    canvas.stroke()


PointEditorCanvas(
    size=(600,600), 
    pts=[PointE2(100, 200), PointE2(200, 200), 
         PointE2(200, 300), PointE2(200, 400), 
         PointE2(300, 200), PointE2(300, 300), 
         PointE2(300, 400), PointE2(400, 400), 
         PointE2(500, 100)], 
    draw_func=draw
)

Canvas(layout=Layout(height='600px', width='600px'), size=(600, 600))

In [5]:
from koebe.geometries.euclidean2 import PointE2
from ipycanvas import Canvas, hold_canvas

canvAnim = Canvas(size=(600,600))
points = perturb([PointE2(100, 200), PointE2(200, 200), 
          PointE2(200, 300), PointE2(200, 400), 
          PointE2(300, 200), PointE2(300, 300), 
          PointE2(300, 400), PointE2(400, 400), 
          PointE2(500, 100)])
selectedPointIdx = -1

frame = 0

def draw():
    global canvAnim, points, selectedPointIdx, frame
    
    theCanvas = canvAnim
    
    with hold_canvas(theCanvas):
        theCanvas.clear()
        
        convHulls, chains = incrConvAnim(points)
        convHull = convHulls[frame % len(convHulls)]
        chain = chains[frame % len(convHulls)]
        
        if frame % len(convHulls) == 0:
            frame = 0
        
        if len(chain) > 0:
            theCanvas.stroke_style = "orange"
            theCanvas.begin_path()
            theCanvas.move_to(round(chain[0].x), round(chain[0].y))
            for v in chain[1:]:
                theCanvas.line_to(round(v.x), round(v.y))
            theCanvas.stroke()
        
        theCanvas.stroke_style = "black"
        
        theCanvas.begin_path()
        theCanvas.move_to(round(convHull.firstVertex.pos.x), round(convHull.firstVertex.pos.y))
        for p in convHull.vertexPositions():
            theCanvas.line_to(round(p.x), round(p.y))
        theCanvas.close_path()
        theCanvas.stroke()
        
        theCanvas.fill_style = 'blue'
        theCanvas.fill_rects([round(p.x) - 4 for p in points], 
                           [round(p.y) - 4 for p in points], 
                           8)
        if selectedPointIdx != -1:
            theCanvas.fill_style = 'orange'
            theCanvas.fill_rect(round(points[selectedPointIdx].x) - 4, 
                              round(points[selectedPointIdx].y) - 4, 
                              8)
        if frame != 0:
            theCanvas.fill_style = 'green'
            theCanvas.fill_rect(round(convHull.firstVertex.pos.x) - 4, 
                                round(convHull.firstVertex.pos.y) - 4,
                               8)

def handle_mouse_down(x, y):
    global points, selectedPointIdx
    # See if any point is close to x, y
    cursorPoint = PointE2(x, y)
    sqDists = [p.distSqTo(cursorPoint) for p in points]
    minIdx = sqDists.index(min(sqDists))
    if sqDists[minIdx] < 24:
        selectedPointIdx = minIdx
    draw()

def handle_mouse_up(x, y):
    global selectedPointIdx
    selectedPointIdx = -1 # No point is selected anymore.
    draw()
    
def handle_mouse_move(x, y):
    global points, selectedPointIdx
    if selectedPointIdx >= 0:
        points[selectedPointIdx] = PointE2(x, y)
    draw()

canvAnim.on_mouse_down(handle_mouse_down)
canvAnim.on_mouse_up(handle_mouse_up)
canvAnim.on_mouse_move(handle_mouse_move)

draw()

import time
import threading

def anim_loop(name):
    global frame
    while True:
        draw()
        time.sleep(1)
        frame += 1

animThread = threading.Thread(target=anim_loop, args=(1,))
animThread.start()

canvAnim

Canvas(layout=Layout(height='600px', width='600px'), size=(600, 600))

In [5]:
from functools import cmp_to_key

def lht_comparator(p):
    def compare(q0, q1):
        area = areaOfParallelogram(p, q0, q1)
        if area < 0:
            return 1
        elif area > 0:
            return -1
        else:
            distSq0 = p.distSqTo(q0)
            distSq1 = p.distSqTo(q1)
            if distSq0 < distSq1:
                return -1
            elif distSq0 > distSq1:
                return 1
            else:
                return 0
    return compare

def grahamScan_sortHelper(P):
    lowest = min(P, key=lambda p: (p.y, p.x))
    otherPoints = sorted([p for p in P if p is not lowest], 
                         key=cmp_to_key(lht_comparator(lowest)))
    return [lowest] + list(otherPoints)
    

def grahamScan(P):
    points = grahamScan_sortHelper(P)
    
    head = Vertex(points[0])
    tail = Vertex(points[1])
    head.setNextVertex(tail)
    i = 0
    for p in points[2:]:
        tail.setNextVertex(Vertex(p))
        tail = tail.next
        while (tail.prev.prev != None and 
               leftHandTurn(tail.pos, tail.prev.pos, tail.prev.prev.pos)):
            tail.prev.prev.setNextVertex(tail)
    tail.setNextVertex(head)
    result = Polygon()
    result.firstVertex = head
    return result
    

In [126]:
pts = grahamScan_sortHelper(points)
head = Vertex(pts[0])
tail = Vertex(pts[1])
head.setNextVertex(tail)
tail.setNextVertex(Vertex(pts[2]))
tail = tail.next
print(leftHandTurn(tail.pos, tail.prev.pos, tail.prev.prev.pos))
print(tail.pos)
print(tail.prev.pos)
print(tail.prev.prev.pos)
print(pts)

False
PointE2(x=298.671875, y=391.65625)
PointE2(x=399.99999999624066, y=400.0000000005195)
PointE2(x=499.9999999990236, y=100.00000000344096)
[PointE2(x=499.9999999990236, y=100.00000000344096), PointE2(x=399.99999999624066, y=400.0000000005195), PointE2(x=298.671875, y=391.65625), PointE2(x=300.00000000470646, y=300.00000000162214), PointE2(x=199.99999999561305, y=400.00000000385825), PointE2(x=199.99999999521305, y=300.0000000043334), PointE2(x=300.00000000266925, y=200.00000000420937), PointE2(x=199.9999999982232, y=200.0000000011201), PointE2(x=100.00000000351903, y=199.9999999954836)]


In [127]:
grahamScan(points).vertexPositions()

[PointE2(x=499.9999999990236, y=100.00000000344096),
 PointE2(x=399.99999999624066, y=400.0000000005195),
 PointE2(x=199.99999999561305, y=400.00000000385825),
 PointE2(x=100.00000000351903, y=199.9999999954836)]

In [6]:
from koebe.geometries.euclidean2 import PointE2
from ipycanvas import Canvas, hold_canvas

canvSorted = Canvas(size=(600,600))
    
points = perturb([PointE2(100, 200), PointE2(200, 200), 
          PointE2(200, 300), PointE2(200, 400), 
          PointE2(300, 200), PointE2(300, 300), 
          PointE2(300, 400), PointE2(400, 400), 
          PointE2(500, 100)])
selectedPointIdx = -1

frame = 0

def draw():
    global canvSorted, points, selectedPointIdx, frame
    
    theCanvas = canvSorted
    
    with hold_canvas(theCanvas):
        theCanvas.clear()
        sortedPoints = grahamScan_sortHelper(points)
        
        height = theCanvas.size[1]
#         convHulls, chains = incrConvAnim(points)
#         convHull = convHulls[frame % len(convHulls)]
#         chain = chains[frame % len(convHulls)]
        
#         if frame % len(convHulls) == 0:
#             frame = 0
        
#         if len(chain) > 0:
#             theCanvas.stroke_style = "red"
#             theCanvas.begin_path()
#             theCanvas.move_to(round(chain[0].x), round(chain[0].y))
#             for v in chain[1:]:
#                 theCanvas.line_to(round(v.x), round(v.y))
#             theCanvas.stroke()
        
        
#         theCanvas.begin_path()
#         theCanvas.move_to(round(convHull.firstVertex.pos.x), round(convHull.firstVertex.pos.y))
#         for p in convHull.vertexPositions():
#             theCanvas.line_to(round(p.x), round(p.y))
#         theCanvas.close_path()
#         theCanvas.stroke()

        convHull = grahamScan(points)
    
    
        ps = Polygon(sortedPoints).vertexPositions()
    
        theCanvas.stroke_style = "#ff0000"
        
        theCanvas.begin_path()
        theCanvas.move_to(round(ps[0].x), height - round(ps[0].y))
        for p in ps:
            theCanvas.line_to(round(p.x), height - round(p.y))
        theCanvas.close_path()
        theCanvas.stroke()
    
        ps = convHull.vertexPositions()
    
        theCanvas.stroke_style = "#00ff00"
        
        theCanvas.begin_path()
        theCanvas.move_to(round(ps[0].x), height - round(ps[0].y))
        for p in ps:
            theCanvas.line_to(round(p.x), height - round(p.y))
        theCanvas.close_path()
        theCanvas.stroke()

        theCanvas.stroke_style = "black"
        
        for p in sortedPoints[1:]:
            theCanvas.begin_path()
            theCanvas.move_to(round(sortedPoints[0].x), height - round(sortedPoints[0].y))
            theCanvas.line_to(round(p.x), height - round(p.y))
            theCanvas.stroke()
        
        theCanvas.fill_style = 'blue'
        theCanvas.fill_rects([round(p.x) - 4 for p in points], 
                           [height - round(p.y) - 4 for p in points], 
                           8)
        
        for pIdx in range(len(sortedPoints)):
            p = sortedPoints[pIdx]
            theCanvas.fill_text(str(pIdx), round(p.x) + 6, height - round(p.y) + 6)
            
        if selectedPointIdx != -1:
            theCanvas.fill_style = 'red'
            theCanvas.fill_rect(round(points[selectedPointIdx].x) - 4, 
                              height - round(points[selectedPointIdx].y) - 4, 
                              8)
#         if frame != 0:
#             theCanvas.fill_style = 'green'
#             theCanvas.fill_rect(round(convHull.firstVertex.pos.x) - 4, 
#                                 round(convHull.firstVertex.pos.y) - 4,
#                                8)

def handle_mouse_down(x, y):
    global points, selectedPointIdx
    y = 600-y
    # See if any point is close to x, y
    cursorPoint = PointE2(x, y)
    sqDists = [p.distSqTo(cursorPoint) for p in points]
    minIdx = sqDists.index(min(sqDists))
    if sqDists[minIdx] < 24:
        selectedPointIdx = minIdx
    draw()

def handle_mouse_up(x, y):
    global selectedPointIdx
    y = 600-y
    selectedPointIdx = -1 # No point is selected anymore.
    draw()
    
def handle_mouse_move(x, y):
    global points, selectedPointIdx
    y = 600-y
    if selectedPointIdx >= 0:
        points[selectedPointIdx] = PointE2(x, y)
    draw()

canvSorted.on_mouse_down(handle_mouse_down)
canvSorted.on_mouse_up(handle_mouse_up)
canvSorted.on_mouse_move(handle_mouse_move)

draw()

# import time
# import threading

# def anim_loop(name):
#     global frame
#     while True:
#         draw()
#         time.sleep(1)
#         frame += 1

# animThread = threading.Thread(target=anim_loop, args=(1,))
# animThread.start()

canvSorted

Canvas(height=600, width=600)